In [1]:
import sys
sys.path.append("/Users/rohan/public_html/Hegemon")
import StepMiner as smn
import HegemonUtil as hu
import re
import numpy as np
import scipy
import io
from scipy import io
import math
import itertools
from itertools import combinations 
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
import statsmodels.stats.proportion
import seaborn as sns
import scanpy as sc
import GEOparse
import os

In [15]:
# Build expr file

directory = "counts"

df = pd.DataFrame()
df["ProbeID"] = 0

gsmdict = {}

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    gsmid = filename.split("_")[0]
    print("processing", gsmid)
    newdf = pd.DataFrame(pd.read_csv(directory+"/"+filename, sep = '\t', header=0))
    
    # If you want header to be GSMID
    #newdf.columns = [gsmid]*len(newdf.columns)
    
    # If you want to keep barcodes as headers and add GSMIDs in ih file
    for i in list(newdf.columns):
        gsmdict[i] = [gsmid]
    
    newdf["ProbeID"] = newdf.index
    df = df.merge(newdf, how="outer", on="ProbeID")

genes = df.pop("ProbeID")
df.insert(0, 'Name', genes)
df.insert(0, 'ProbeID', genes)
df.replace(to_replace=math.nan, value=0, inplace=True) # Deal with missing values
df

processing GSM4648437
processing GSM4648439
processing GSM4648438


,ProbeID,Name,AAACGAAAGTGAGTGC_Lung.OVA.SIA,AAACGAACAAGCGGAT_Lung.OVA.SIA,AAACGCTCACGGTGTC_Lung.OVA.SIA,AAAGAACCAGAATTCC_Lung.OVA.SIA,AAAGGATGTTGTCATG_Lung.OVA.SIA,AAAGGATTCAGTCACA_Lung.OVA.SIA,AAAGTCCCACATATGC_Lung.OVA.SIA,AACAAAGCAGCAGACA_Lung.OVA.SIA,...,TTGTGTTCATCCAACA_Synovium.OVA.SIA,TTGTTGTTCCATTCGC_Synovium.OVA.SIA,TTTACCAAGTTCACTG_Synovium.OVA.SIA,TTTACCATCGTTATCT_Synovium.OVA.SIA,TTTACGTTCTGAGAAA_Synovium.OVA.SIA,TTTATGCCATGACGTT_Synovium.OVA.SIA,TTTCACACAAATCAAG_Synovium.OVA.SIA,TTTCCTCAGGCTGAAC_Synovium.OVA.SIA,TTTGATCAGACCAACG_Synovium.OVA.SIA,TTTGATCCAGCCGTCA_Synovium.OVA.SIA
0,Mrpl15,Mrpl15,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Lypla1,Lypla1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2,Tcea1,Tcea1,0.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0
3,Atp6v1h,Atp6v1h,0.0,1.0,7.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Rb1cc1,Rb1cc1,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13435,Crtac1,Crtac1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13436,Cpn1,Cpn1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13437,Psd,Psd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13438,Nutf2-ps1,Nutf2-ps1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
# Normalize and log transform expression values

expr = df.copy(deep=True)
expr = expr.drop(['ProbeID', 'Name'], axis=1)

print("Taking log")
adata = sc.AnnData(expr.T)
sc.pp.normalize_total(adata, target_sum=1e6)
sc.pp.log1p(adata, base = 2)

expr = pd.DataFrame(adata.X)
expr = expr.T

genes = pd.DataFrame()
genes["ProbeID"] = df["ProbeID"]
genes["Name"] = df["Name"]
expr = pd.concat([genes, expr], axis=1)
expr.columns = df.columns
expr

Taking log


/usr/local/lib/python3.6/dist-packages/anndata/_core/anndata.py:119: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


,ProbeID,Name,AAACGAAAGTGAGTGC_Lung.OVA.SIA,AAACGAACAAGCGGAT_Lung.OVA.SIA,AAACGCTCACGGTGTC_Lung.OVA.SIA,AAAGAACCAGAATTCC_Lung.OVA.SIA,AAAGGATGTTGTCATG_Lung.OVA.SIA,AAAGGATTCAGTCACA_Lung.OVA.SIA,AAAGTCCCACATATGC_Lung.OVA.SIA,AACAAAGCAGCAGACA_Lung.OVA.SIA,...,TTGTGTTCATCCAACA_Synovium.OVA.SIA,TTGTTGTTCCATTCGC_Synovium.OVA.SIA,TTTACCAAGTTCACTG_Synovium.OVA.SIA,TTTACCATCGTTATCT_Synovium.OVA.SIA,TTTACGTTCTGAGAAA_Synovium.OVA.SIA,TTTATGCCATGACGTT_Synovium.OVA.SIA,TTTCACACAAATCAAG_Synovium.OVA.SIA,TTTCCTCAGGCTGAAC_Synovium.OVA.SIA,TTTGATCAGACCAACG_Synovium.OVA.SIA,TTTGATCCAGCCGTCA_Synovium.OVA.SIA
0,Mrpl15,Mrpl15,6.321054,0.000000,6.622387,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
1,Lypla1,Lypla1,0.000000,0.000000,6.622387,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,9.931639,0.000000,0.000000,0.0,0.000000,0.000000,0.0,6.921326
2,Tcea1,Tcea1,0.000000,8.241983,8.611363,0.000000,0.0,0.0,0.0,0.0,...,0.0,7.522376,0.000000,0.000000,0.000000,0.0,9.612612,0.000000,0.0,5.933180
3,Atp6v1h,Atp6v1h,0.000000,8.241983,9.417135,6.057254,0.0,0.0,0.0,0.0,...,0.0,7.522376,0.000000,6.327367,0.000000,0.0,0.000000,0.000000,0.0,0.000000
4,Rb1cc1,Rb1cc1,0.000000,0.000000,8.197555,6.057254,0.0,0.0,0.0,0.0,...,0.0,7.522376,0.000000,0.000000,8.734062,0.0,0.000000,6.862206,0.0,6.921326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13435,Crtac1,Crtac1,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
13436,Cpn1,Cpn1,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
13437,Psd,Psd,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
13438,Nutf2-ps1,Nutf2-ps1,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000


In [48]:
# If you want ih file to have barcodes
gsmdf =  pd.DataFrame(gsmdict).T
gsmdf.columns = ["ArrayHeader"]
gsmdf["ArrayID"] = gsmdf.index
gsmdf.reset_index(inplace=True)
gsmdf.drop("index",axis=1,inplace=True)
gsmdf

# Fix the ih file generated from process.py
ihdf = pd.DataFrame(pd.read_csv("GSE153632-GPL17021-old-ih.txt", sep = '\t', header=0))
ihdf.drop("ArrayID", axis=1, inplace=True)
ihdf = gsmdf.merge(ihdf, how="right", on="ArrayHeader")
ihdf = ihdf[["ArrayID", "ArrayHeader", "ClinicalHeader"]]
ihdf

,ArrayID,ArrayHeader,ClinicalHeader
0,AAACGAAAGTGAGTGC_Lung.OVA.SIA,GSM4648437,Lung_OVA+SIA
1,AAACGAACAAGCGGAT_Lung.OVA.SIA,GSM4648437,Lung_OVA+SIA
2,AAACGCTCACGGTGTC_Lung.OVA.SIA,GSM4648437,Lung_OVA+SIA
3,AAAGAACCAGAATTCC_Lung.OVA.SIA,GSM4648437,Lung_OVA+SIA
4,AAAGGATGTTGTCATG_Lung.OVA.SIA,GSM4648437,Lung_OVA+SIA
...,...,...,...
1386,TTGGATGTCCGGACTG_Synovium.SIA,GSM4648439,Synovium_SIA
1387,TTGTTTGAGTTCGGTT_Synovium.SIA,GSM4648439,Synovium_SIA
1388,TTTACCACATGACAGG_Synovium.SIA,GSM4648439,Synovium_SIA
1389,TTTGTTGAGGCCACCT_Synovium.SIA,GSM4648439,Synovium_SIA


In [56]:
# If you want survival file to have barcodes
gsmdf.columns = ["ArrayId", "ArrayID"]
# Read survival file from process.py and modify it
survival = pd.DataFrame(pd.read_csv("GSE153632-GPL17021-survival.txt", sep = '\t', header=0))
survival = gsmdf.merge(survival, how="right", on="ArrayId")
survival.drop("ArrayId", axis=1, inplace=True)
survival

,ArrayID,time,status,c title,c source_name (ch1),c organism (ch1),c taxid (ch1),c tissue (ch1),c strain (ch1),c cell type (ch1),c treatment (ch1),c time point (ch1),c molecule (ch1)
0,AAACGAAAGTGAGTGC_Lung.OVA.SIA,NaN,NaN,Lung_OVA+SIA,Lung eosinophil granulocytes,Mus musculus,10090,Lung tissue,Balb/c,CD45+CD11b+Siglec-F+ granulocytes,OVA+SIA,Day 7 post SIA,total RNA
1,AAACGAACAAGCGGAT_Lung.OVA.SIA,NaN,NaN,Lung_OVA+SIA,Lung eosinophil granulocytes,Mus musculus,10090,Lung tissue,Balb/c,CD45+CD11b+Siglec-F+ granulocytes,OVA+SIA,Day 7 post SIA,total RNA
2,AAACGCTCACGGTGTC_Lung.OVA.SIA,NaN,NaN,Lung_OVA+SIA,Lung eosinophil granulocytes,Mus musculus,10090,Lung tissue,Balb/c,CD45+CD11b+Siglec-F+ granulocytes,OVA+SIA,Day 7 post SIA,total RNA
3,AAAGAACCAGAATTCC_Lung.OVA.SIA,NaN,NaN,Lung_OVA+SIA,Lung eosinophil granulocytes,Mus musculus,10090,Lung tissue,Balb/c,CD45+CD11b+Siglec-F+ granulocytes,OVA+SIA,Day 7 post SIA,total RNA
4,AAAGGATGTTGTCATG_Lung.OVA.SIA,NaN,NaN,Lung_OVA+SIA,Lung eosinophil granulocytes,Mus musculus,10090,Lung tissue,Balb/c,CD45+CD11b+Siglec-F+ granulocytes,OVA+SIA,Day 7 post SIA,total RNA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1386,TTGGATGTCCGGACTG_Synovium.SIA,NaN,NaN,Synovium_SIA,Synovial eosinophil granulocytes,Mus musculus,10090,Hind paw synovial tissue,Balb/c,CD45+CD11b+Siglec-F+ granulocytes,SIA,Day 7 post SIA,total RNA
1387,TTGTTTGAGTTCGGTT_Synovium.SIA,NaN,NaN,Synovium_SIA,Synovial eosinophil granulocytes,Mus musculus,10090,Hind paw synovial tissue,Balb/c,CD45+CD11b+Siglec-F+ granulocytes,SIA,Day 7 post SIA,total RNA
1388,TTTACCACATGACAGG_Synovium.SIA,NaN,NaN,Synovium_SIA,Synovial eosinophil granulocytes,Mus musculus,10090,Hind paw synovial tissue,Balb/c,CD45+CD11b+Siglec-F+ granulocytes,SIA,Day 7 post SIA,total RNA
1389,TTTGTTGAGGCCACCT_Synovium.SIA,NaN,NaN,Synovium_SIA,Synovial eosinophil granulocytes,Mus musculus,10090,Hind paw synovial tissue,Balb/c,CD45+CD11b+Siglec-F+ granulocytes,SIA,Day 7 post SIA,total RNA


In [50]:
# Export files

path_dir = "./"
expr.to_csv(path_dir+"GSE153632-GPL17021-expr.txt", header=True, index=False,sep='\t')
ihdf.to_csv(path_dir+"GSE153632-GPL17021-ih.txt", header=True, index=False,sep='\t')
survival.to_csv(path_dir+"GSE153632-GPL17021-survival.txt", header=True, index=False,sep='\t')

In [51]:
# Build idx file

def make_idx():
    print('Starting make_idx')
    expr = path_dir+'GSE153632-GPL17021-expr.txt'

    ptr = []
    ids = []
    name = []
    desc = []
    pos = 0

    with open(expr, 'rb') as f:
        for line in f:
            if pos == 0:
                pos += len(line)
            else:
                ptr.append(pos)
                pos += len(line)
                split = line.decode("utf-8").split('\t')
                ids.append(split[0])
                name.append(split[1].split(':')[0])
                desc.append(':'.join(split[1].split(':')[1:]))
        f.close()

    with open(path_dir+'GSE153632-GPL17021-idx.txt', 'w') as f:
        f.write('ProbeID\tPtr\tName\tDescription\n')
        for i in range(len(ids)):
            f.write('{}\t{}\t{}\t{}\n'.format(ids[i], ptr[i], name[i], desc[i]))
        f.close()
    print("Done with make_idx")
    
make_idx()

Starting make_idx
Done with make_idx
